## Deep Learning Model Evaluation

- Almost always, we are interested to plot the training accuracy and testing (validation set) accuracy over epochs

- Also, we are interested to see the loss is decreasing over epochs

### How to plot accuracy and loss over epochs

In [ ]:
import matplotlib.pyplot as plt

# use 70% for train, 30% for validation
history = model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, verbose=0) 

# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')     # y label
plt.xlabel('epoch')     # x label
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Activity: Plot the model accuracy for training set, model accuracy for validation set

- Also plot model loss for both training and validation set for MNIST MLP model 

- Hint:
    - Hidden Layer 1: Fully Connected + Relu Activition (e.g. 512 Nuerons)
    - Hidden Layer 2: Fully Connected + Relu Activition (e.g. 512 Neurons)
    - Outout Layer: Fully Connected + Softmax Activition
    - Set epoch = 10

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import keras
import matplotlib.pyplot as plt #This package is for plotting
%matplotlib inline  
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD
from keras.initializers import RandomNormal
from keras.models import load_model


(x_train, y_train), (x_test, y_test) = mnist.load_data()    # building mnist

In [3]:
x_train = x_train/np.max(x_train)     # building data
x_test = x_test/np.max(x_test)

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [4]:
x_train = x_train.reshape(x_train.shape[0], 28, 28)    # number of samples in train: x_train.shape[0], 28x28 pixels4
x_test = x_test.reshape(x_test.shape[0], 28, 28)

In [6]:
model = Sequential()

# hidden layer 1
model.add(Dense(512, activation='relu', input_shape=(784,)))
# hidden layer 2
model.add(Dense(512, activation='relu'))
# output layer
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics = ['accuracy'])

# use 70% for train, 30% for validation
history = model.fit(x_train,
                    y_train,
                    epochs=10,
                    batch_size=128,
                    verbose=1,
                    validation_split=0.3)

ValueError: Error when checking input: expected dense_4_input to have 2 dimensions, but got array with shape (60000, 28, 28)

In [ ]:
# list all data in history
print(history.history.keys())

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')     # y label
plt.xlabel('epoch')     # x label
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Callbacks in Keras

- Callbacks are functions that can be applied at certain stages of the training process, such as at the end of each epoch

- Specifically, in our solution, we included `EarlyStopping(monitor='val_loss', patience=2)` to define that we wanted to monitor the validation loss at each epoch and after the validation loss has not improved after two epochs, training is interrupted

- However, since we set patience=2, we won’t get the best model, but the model two epochs after the best model. Therefore, optionally, we can include a second operation, `ModelCheckpoint` which saves the model to a file after every checkpoint



<img src = 'https://github.com/Make-School-Courses/DS-2.2-Deep-Learning/raw/master/Notebooks/Images/early_stopping.png'>

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Set callback functions to early stop training and save the best model so far
callbacks = [EarlyStopping(monitor='val_loss', patience=2),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

In [ ]:
# Train neural network
history = model.fit(train_features, # Features
                      train_target, # Target vector
                      epochs=20, # Number of epochs
                      callbacks=callbacks, # Early stopping
                      verbose=0, # Print description after each epoch
                      batch_size=100, # Number of observations per batch
                      validation_data=(test_features, test_target)) # Data for evaluation

### Tensorboard

- It hosts a website on your local machine in which you can monitor things like accuracy, cost functions and visualize the computational graph that Tensorflow is running based on what you defined in Keras


- For monitoring progress, open the terminal: `tensorboard --logdir Graph`

In [ ]:
from keras.callbacks import TensorBoard

tensor_board = TensorBoard(log_dir='./Graph')

model.fit(x_train, y_train, verbose=1, callbacks=[tensor_board])

### Save and Load Keras Model

- Given that deep learning models can take hours, days, or weeks to train, it is paramount to know how to save and load them from disk

- Option 1: Weights + Model Architecture

- Option 2: Save/Load the Entire Model

In [ ]:
from keras.models import model_from_json

# Option 1: Save Weights + Architecture
model.save_weights('model_weights.h5')
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())
    
# Option 1: Load Weights + Architecture
with open('model_architecture.json', 'r') as f:
    new_model_1 = model_from_json(f.read())
new_model_1.load_weights('model_weights.h5')

In [ ]:
# Option 2:

from keras.models import load_model

# Option 2: Save/load the entire model

# Creates a HDF5 file 'my_model.h5'
model.save('my_model.h5')

# Deletes the existing model
del model  

# Returns a compiled model identical to the previous one
model = load_model('my_model.h5')

# Activity: Train your MLP model for MNIST and save the model after training with Option 1 and Option 2. 

- Compare the size of model's weights file + model's architecture size from Option 1 with model's structure from Option 2

In [ ]:
# SVG
from keras.utils import plot_model

plot_model(model, to_file='model.png', show_shapes=True)